Notebook para a divisão k-fold

In [31]:
import pandas as pd

In [32]:
import numpy as np

In [33]:
def read_file_as_df(file_name):
    import pandas as pd
    import csv

    import sys
    import pandas as pd

    maxInt = sys.maxsize

    while True:
        # decrease the maxInt value by factor 10 
        # as long as the OverflowError occurs.

        try:
            csv.field_size_limit(maxInt)
            break
        except OverflowError:
            maxInt = int(maxInt/10)

    file = []
    col = []

    with open(file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        line_count = 0
        for row in csv_reader:
            if line_count==0:
                for r in row:
                    col.append(r)
                line_count+=1
            else:
                line = []
                for r in row:
                    line.append(r)
                file.append(line)
                line_count += 1

            
    df = pd.DataFrame(file, columns = col)
    return df


In [34]:
df = read_file_as_df('brMoral.csv')

In [35]:
df.dropna(axis=1, inplace=True)
df.drop_duplicates(inplace=True)

In [36]:
df.shape

(510, 77)

In [37]:
df = df.drop(columns=['t.gay-marriage', 't.gun-control', 't.abortion',
       't.death-penalty', 't.drugs', 't.criminal-age', 't.racial-quotas',
       't.church-tax', 'school', 'it',
       'religion', 'politics', 'age', 'ap.school', 'ap.religion',
       'ap.politics', 's.gay-marriage', 's.gun-control', 's.abortion',
       's.death-penalty', 's.drugs', 's.criminal-age', 's.racial-quotas',
       's.church-tax', 'st.gay-marriage', 'st.gun-control', 'st.abortion',
       'st.death-penalty', 'st.drugs', 'st.criminal-age', 'st.racial-quotas',
       'st.church-tax', 'care', 'fairness', 'loyalty', 'authority', 'purity',
       'mf.care', 'mf.fairness', 'mf.loyalty', 'mf.authority', 'mf.purity',
       'item1', 'item2', 'item3', 'item4', 'item5', 'item7', 'item8', 'item9',
       'item10', 'item11', 'item12', 'item13', 'item14', 'item15', 'item16',
       'item17', 'item18', 'item19', 'item20', 'item21', 'item23', 'item24',
       'item25', 'item26', 'item27', 'item28', 'item29', 'item30', 'item31',
       'item32'])

In [38]:
df.columns

Index(['sid', 'concat', 'freetext', 'gender', 'ap.age'], dtype='object')

In [39]:
df.shape

(510, 5)

In [40]:
def age_groups(text):
    if text=='a0-23':
        return 1
    elif text=='a24-30':
        return 2
    else:
        return 3

In [41]:
df["AgeClass"] = df["ap.age"].apply(age_groups)

In [42]:
df.head()

,sid,concat,freetext,gender,ap.age,AgeClass
0,sid6000,"Não vejo e nem nunca vi problema nisso, acho q...",A flexibilidade é extremamente importante nas ...,m,a0-23,1
1,sid6001,Acredito que todos possuem direito de escolher...,Acabo de responder a perguntas bastante polêmi...,m,a24-30,2
2,sid959,Acredito que todos tem o direito de escolher c...,"Achei esse teste cansativo à beça, mas me fez ...",m,a24-30,2
3,sid842,Vejo que as pessoas tem todo o direito de esco...,amanhã trabalho cansaço tarde engenharia dados...,m,a0-23,1
4,sid774,As pessoas devem ser livres para tomar decisõe...,Algo difícil na vida é conquistar sua independ...,f,a0-23,1


In [43]:
def genderGroups(text):
    if(text == "f"):
        return 1
    else:
        return 0

In [44]:
df["GenderClass"] = df["gender"].apply(genderGroups)

In [45]:
def GroupAgeAndGender(df):
    ageGender = []
    for _, line in df.iterrows():
        if line["AgeClass"]==1 and line["GenderClass"]==1:
            ageGender.append(0)
        elif line["AgeClass"]==1 and line["GenderClass"]==0:
            ageGender.append(1)
        elif line["AgeClass"]==2 and line["GenderClass"]==1:
            ageGender.append(2)
        elif line["AgeClass"]==2 and line["GenderClass"]==0:
            ageGender.append(3)
        elif line["AgeClass"]==3 and line["GenderClass"]==1:
            ageGender.append(4)
        elif line["AgeClass"]==3 and line["GenderClass"]==0:
            ageGender.append(5)
    df["AgeGender"] = ageGender

In [46]:
GroupAgeAndGender(df)

In [47]:
df.reset_index(drop=True, inplace=True)

In [48]:
from sklearn.model_selection import StratifiedKFold

In [49]:
kf_strat = StratifiedKFold(n_splits=10, shuffle=True)

In [50]:
x = df
y = df["AgeGender"]

In [51]:
def getsplits_strat(df, samples):
     kfold_list = []
     for train_index, test_index in kf_strat.split(df, samples):
          print(len(train_index), len(test_index))
          dftrain = df.iloc[train_index]
          dftest = df.iloc[test_index]
          kfold_list.append([dftrain, dftest])
     return kfold_list

In [52]:
kfold_list = getsplits_strat(x, y)

459 51
459 51
459 51
459 51
459 51
459 51
459 51
459 51
459 51
459 51


In [53]:
for train, test in kfold_list:
    train_fem = train[train["GenderClass"]==1]
    train_masc = train[train["GenderClass"]==0]
    print(train_fem["AgeClass"].value_counts())
    print(train_masc["AgeClass"].value_counts())

    test_fem = test[test["GenderClass"]==1]
    test_masc = test[test["GenderClass"]==0]
    print(test_fem["AgeClass"].value_counts())
    print(test_masc["AgeClass"].value_counts())

1    59
3    53
2    46
Name: AgeClass, dtype: int64
2    118
1    109
3     74
Name: AgeClass, dtype: int64
1    6
2    5
3    5
Name: AgeClass, dtype: int64
2    13
1    13
3     9
Name: AgeClass, dtype: int64
1    59
3    53
2    45
Name: AgeClass, dtype: int64
2    118
1    109
3     75
Name: AgeClass, dtype: int64
1    6
2    6
3    5
Name: AgeClass, dtype: int64
1    13
2    13
3     8
Name: AgeClass, dtype: int64
1    59
3    52
2    46
Name: AgeClass, dtype: int64
2    117
1    110
3     75
Name: AgeClass, dtype: int64
1    6
3    6
2    5
Name: AgeClass, dtype: int64
2    14
1    12
3     8
Name: AgeClass, dtype: int64
1    58
3    52
2    46
Name: AgeClass, dtype: int64
2    118
1    110
3     75
Name: AgeClass, dtype: int64
1    7
3    6
2    5
Name: AgeClass, dtype: int64
2    13
1    12
3     8
Name: AgeClass, dtype: int64
1    58
3    52
2    46
Name: AgeClass, dtype: int64
2    118
1    110
3     75
Name: AgeClass, dtype: int64
1    7
3    6
2    5
Name: AgeClass, dtype:

### Salvar em arquivo

In [54]:
def create_split_csv(l, path):
    import os
    import shutil

    root_folder = path
    folder_name = "k_"
    file_name1 = "train"
    file_name2 = "test"
    k = 1

    for training, test in l:
        path = os.path.join(root_folder, (folder_name+str(k)))
        if(os.path.exists(path)):
            shutil.rmtree(path)
        os.mkdir(path)

        training.to_csv((path+"/"+file_name1+".csv"), index = False, header=True)
        test.to_csv((path+"/"+file_name2+".csv"), index = False, header=True)

        k+=1

In [55]:
create_split_csv(kfold_list, "C:/Users/alici/Documents/tcc/github2/tcc-v2/brmoral/particoes")